In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql import functions as F
import os

os.environ['PYSPARK_PYTHON'] = '/usr/local/bin/python3.9'

spark = SparkSession.builder.appName("Spark_sql_test").\
    master("local[*]").\
    getOrCreate()

sc = spark.sparkContext

your 131072x1 screen size is bogus. expect trouble


24/04/27 18:02:20 WARN Utils: Your hostname, SIMON_WANG resolves to a loopback address: 127.0.1.1; using 172.26.38.52 instead (on interface eth0)
24/04/27 18:02:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/27 18:02:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.format('csv').\
    option("header", "true").\
    load("car_prices.csv")
df.printSchema()
df.show()

df.createOrReplaceTempView("car_prices")

root
 |-- year: string (nullable = true)
 |-- make: string (nullable = true)
 |-- model: string (nullable = true)
 |-- trim: string (nullable = true)
 |-- body: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- vin: string (nullable = true)
 |-- state: string (nullable = true)
 |-- condition: string (nullable = true)
 |-- odometer: string (nullable = true)
 |-- color: string (nullable = true)
 |-- interior: string (nullable = true)
 |-- seller: string (nullable = true)
 |-- mmr: string (nullable = true)
 |-- sellingprice: string (nullable = true)
 |-- saledate: string (nullable = true)

+----+---------+-------------------+--------------------+-----------+------------+-----------------+-----+---------+--------+------+--------+--------------------+-------+------------+--------------------+
|year|     make|              model|                trim|       body|transmission|              vin|state|condition|odometer| color|interior|              seller|    mmr|selling

In [3]:
# 年份统计
# show(n=30) n表示最多展示的行数
df.select('*').groupBy('year').count().sort('year',ascending=False).show(n=30)

+----+-----+
|year|count|
+----+-----+
|2015| 7994|
|2014|69712|
|2013|87467|
|2012|87380|
|2011|41384|
|2010|22616|
|2009|17959|
|2008|27011|
|2007|25378|
|2006|21631|
|2005|17169|
|2004|13624|
|2003|10368|
|2002| 7693|
|2001| 5140|
|2000| 3427|
|1999| 2227|
|1998| 1464|
|1997| 1042|
|1996|  562|
|1995|  483|
|1994|  286|
|1993|  127|
|1992|   97|
|1991|   51|
|1990|   33|
+----+-----+



In [4]:
# make统计

df.select('*').groupBy('make').count().sort('count',ascending=False).show(n=30)

+-------------+-----+
|         make|count|
+-------------+-----+
|         Ford|81013|
|    Chevrolet|54150|
|       Nissan|44043|
|       Toyota|35313|
|        Dodge|27181|
|        Honda|24781|
|      Hyundai|18659|
|          BMW|17509|
|          Kia|15828|
|     Chrysler|15133|
|     Infiniti|14011|
|Mercedes-Benz|14008|
|         Jeep|12735|
|   Volkswagen|10780|
|        Lexus|10409|
|          GMC| 9217|
|        Mazda| 6930|
|     Cadillac| 6315|
|      Lincoln| 4926|
|         Audi| 4802|
|        Acura| 4503|
|        Buick| 4470|
|       Subaru| 4369|
|          Ram| 4044|
|      Pontiac| 4018|
|   Mitsubishi| 3661|
|        Volvo| 3082|
|         MINI| 2946|
|       Saturn| 2570|
|      Mercury| 1571|
+-------------+-----+
only showing top 30 rows



In [5]:
# 车型统计信息

sql = '''
SELECT make, model, count(*) as total
FROM car_prices
GROUP BY make, model
ORDER BY make ASC, model DESC
'''

spark.sql(sqlQuery=sql).show(n=50)

Py4JError: An error occurred while calling o25.sql. Trace:
py4j.Py4JException: Method sql([class java.lang.String, class java.util.HashMap]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.lang.Thread.run(Thread.java:750)



In [ ]:
# 最好卖的车型top

sql = '''
SELECT make, model, count(*) as total
FROM car_prices
GROUP BY make, model
ORDER BY total DESC
'''

spark.sql(sqlQuery=sql).show(n=50)

+-------------+----------------+-----+
|         make|           model|total|
+-------------+----------------+-----+
|       Nissan|          Altima|16346|
|         Ford|          Fusion|12116|
|         Ford|           F-150|11950|
|       Toyota|           Camry|10986|
|         Ford|          Escape|10656|
|         Ford|           Focus| 9547|
|        Honda|          Accord| 8528|
|    Chevrolet|          Impala| 7500|
|          BMW|        3 Series| 7132|
|        Honda|           Civic| 7068|
|        Dodge|   Grand Caravan| 6966|
|     Infiniti|         G Sedan| 6939|
|       Toyota|         Corolla| 6745|
|         Ford|        Explorer| 6242|
|    Chevrolet|          Malibu| 6213|
|      Hyundai|          Sonata| 5787|
|    Chevrolet|  Silverado 1500| 5685|
|       Nissan|          Maxima| 5470|
|    Chevrolet|           Cruze| 5404|
|      Hyundai|         Elantra| 5343|
|         Ford|            Edge| 5098|
|     Chrysler|Town and Country| 5053|
|         Ford|         M